In [10]:
import sys
import os
import pandas as pd

path = os.path.abspath('../..')
if path not in sys.path:
    sys.path.insert(0, path)
    
sys.path

from primer3plus.utils import reverse_complement as rc
import primer3
from aqbt.contrib.uwbf import primer_utils
from aqbt import AquariumBuildTools
aqtools = AquariumBuildTools.from_toml('creds.secret.toml')
aqtools.sessions

aq = aqtools.sessions['production']['aquarium']

In [47]:
opids = [ja.operation_id for ja in aq.JobAssociation.where({'job_id': 124856})]

ops = aq.Operation.find(opids)

for op in ops:
    print(op.status)

done
done
done
done
error
done
error
done
done


In [53]:
op.field_values[0].item.object_type_id

865

In [68]:
aq.Operation.find(347510).outputs[0].allowable_field_type.object_type_id

347

In [89]:
aq.FieldType.where({'parent_id': })

{'part': False,
 'preferred_operation_type_id': None,
 'ftype': 'sample',
 'choices': None,
 'parent_class': 'OperationType',
 'preferred_field_type_id': None,
 'id': 23783,
 'updated_at': '2021-04-02T13:13:52.000-07:00',
 'role': 'input',
 'created_at': '2021-04-02T13:13:52.000-07:00',
 'name': 'Overnight',
 'array': False,
 'required': None,
 'routing': 'YS',
 'parent_id': 756,
 'rid': 11407}

In [115]:
aq.Operation.find(347828).operation_type.name

'Inoculate Yeast Comp Cells'

In [109]:
ops[0].operation_type_id = 756

ops[0].plan

from pydent import Planner

planner = Planner(aq.Plan.find(ops[0].plan.id))

for op in planner.operations:
    op.operation_type_id = 756
    for fv in op.field_values:
        ft = aq.FieldType.where({'name': fv.name, 'parent_id': op.operation_type_id, 'parent_class': 'OperationType'})[0]
        fv.field_type_id = ft.id
        fv.field_type = ft
        
        
        if fv.item:
            fv.allowable_field_type_id = aq.AllowableFieldType(
                {'object_type_id': fv.item.object_type_id, 
                 'sample_type_id': fv.item.sample.sample_type_id, 
                 'field_type_id': fv.field_type_id}
            ).id
        else:
            fv.allowable_field_type_id = aq.AllowableFieldType({
                'object_type_id': ft.allowable_field_types[0].object_type_id,
                'sample_type_id': ft.allowable_field_types[0].sample_type_id
            }).id

planner.save()
planner.url

'http:/52.27.43.242/plans?plan_id=41094'

In [9]:
from tqdm.auto import tqdm

samples = aq.Sample.where("user_id = 66 AND sample_type_id = {}".format(
aq.SampleType.find_by_name("Fragment").id))
to_update = []
for s in samples:
    if ' (v2)' in s.name:
        to_update.append(s)
        
for s in tqdm(to_update):
    print(s.name)
    s.update_properties({'Template': s})
    s.save()

  0%|          | 0/18 [00:00<?, ?it/s]

URA-URA<pGRR-W20W36-yeGFP> (v2)
URA-URA<pGRR-W17W17-yeGFP> (v2)
URA-URA<pGRR-W36W36-yeGFP> (v2)
URA-URA<pGRR-W5W36-yeGFP> (v2)
URA-URA<pGRR-W36W5-yeGFP> (v2)
URA-URA<pGRR-W10W20-yeGFP> (v2)
URA-URA<pGRR-W10W17-yeGFP> (v2)
URA-URA<pGRR-W5W8-URGR-W36> (v2)
URA-URA<pGRR-W10W8-URGR-W5> (v2)
LEU-LEU<pGRR-W5W5-URGR-W36> (v2)
ARS314-bleo<pGRR-W10W8-URGR-W36> (v2)
URA-URA<pGRR-W8W34-URGR-W10> (v2)
ARS314-bleo<pGRR-W8W34-URGR-W17> (v2)
LEU-LEU<pGRR-W17W17-URGR-W20> (v2)
URA-URA<pGRR-W8W34-URGR-W17> (v2)
ARS314-bleo<pGRR-W5W5-URGR-W20> (v2)
ARS314-bleo<pGRR-W17W17-URGR-W10> (v2)
URA-URA<pGRR-W17W34-yeGFP> (v2)
